In [1]:
import pandas as pd
import numpy as np
import kagglehub
import os
import matplotlib.pyplot as plt
import seaborn as sns
import random
from sklearn.metrics.pairwise import cosine_similarity
from mlxtend.frequent_patterns import apriori, association_rules
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
path = kagglehub.dataset_download("senapatirajesh/netflix-tv-shows-and-movies")

print("Path to dataset files:", path)
print("Path to dataset files:", path)
csv_file = None
for file in os.listdir(path):
    if file.endswith('.csv'):
        csv_file = os.path.join(path, file)
        break

if csv_file is None:
    print("No CSV file found in the directory.")
else:
    print(f"Found CSV file: {csv_file}")


Path to dataset files: C:\Users\keert\.cache\kagglehub\datasets\senapatirajesh\netflix-tv-shows-and-movies\versions\1
Path to dataset files: C:\Users\keert\.cache\kagglehub\datasets\senapatirajesh\netflix-tv-shows-and-movies\versions\1
Found CSV file: C:\Users\keert\.cache\kagglehub\datasets\senapatirajesh\netflix-tv-shows-and-movies\versions\1\NetFlix.csv


In [3]:
df = pd.read_csv(csv_file, delimiter=',')
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genres,description
0,s1,TV Show,3%,NaN,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,14-Aug-20,2020,TV-MA,4,"International TV Shows, TV Dramas, TV Sci-Fi &...",In a future where the elite inhabit an island ...
1,s10,Movie,1920,Vikram Bhatt,"Rajneesh Duggal, Adah Sharma, Indraneil Sengup...",India,15-Dec-17,2008,TV-MA,143,"Horror Movies, International Movies, Thrillers",An architect and his wife move into a castle t...
2,s100,Movie,3 Heroines,Iman Brotoseno,"Reza Rahadian, Bunga Citra Lestari, Tara Basro...",Indonesia,05-Jan-19,2016,TV-PG,124,"Dramas, International Movies, Sports Movies",Three Indonesian women break records by becomi...
3,s1000,Movie,Blue Mountain State: The Rise of Thadland,Lev L. Spiro,"Alan Ritchson, Darin Brooks, James Cade, Rob R...",United States,01-Mar-16,2016,R,90,Comedies,New NFL star Thad buys his old teammates' belo...
4,s1001,TV Show,Blue Planet II,NaN,David Attenborough,United Kingdom,03-Dec-18,2017,TV-G,1,"British TV Shows, Docuseries, Science & Nature TV",This sequel to the award-winning nature series...


In [4]:
df.drop('description',axis=1,inplace=True)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      5398 non-null   object
 4   cast          7069 non-null   object
 5   country       7280 non-null   object
 6   date_added    7777 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7780 non-null   object
 9   duration      7787 non-null   int64 
 10  genres        7787 non-null   object
dtypes: int64(2), object(9)
memory usage: 669.3+ KB


In [6]:
df.isnull().sum()

show_id            0
type               0
title              0
director        2389
cast             718
country          507
date_added        10
release_year       0
rating             7
duration           0
genres             0
dtype: int64

<h1>Handling Null Values</h1> Replaced ['director','cast','country'] Null values with Unknown,Replace 'date_added' with 'Not Available',Replace 'rating' with mode

In [7]:
df['director']= df['director'].fillna('Unknown')
df['cast']= df['cast'].fillna('Unknown')
df['country'] = df['country'].fillna('Unknown')

In [8]:
df['date_added']=df['date_added'].fillna('Not Available')
df['rating']=df['rating'].fillna(df['rating'].mode()[0])

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7787 entries, 0 to 7786
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       7787 non-null   object
 1   type          7787 non-null   object
 2   title         7787 non-null   object
 3   director      7787 non-null   object
 4   cast          7787 non-null   object
 5   country       7787 non-null   object
 6   date_added    7787 non-null   object
 7   release_year  7787 non-null   int64 
 8   rating        7787 non-null   object
 9   duration      7787 non-null   int64 
 10  genres        7787 non-null   object
dtypes: int64(2), object(9)
memory usage: 669.3+ KB


In [10]:
df['show_id'].value_counts()

show_id
s999     1
s1       1
s10      1
s100     1
s1000    1
        ..
s1010    1
s1011    1
s1012    1
s1013    1
s1014    1
Name: count, Length: 7787, dtype: int64

In [11]:
df['show_id']= df['show_id'].str.replace('s', '').astype(int)

In [12]:
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genres
0,1,TV Show,3%,Unknown,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,14-Aug-20,2020,TV-MA,4,"International TV Shows, TV Dramas, TV Sci-Fi &..."
1,10,Movie,1920,Vikram Bhatt,"Rajneesh Duggal, Adah Sharma, Indraneil Sengup...",India,15-Dec-17,2008,TV-MA,143,"Horror Movies, International Movies, Thrillers"
2,100,Movie,3 Heroines,Iman Brotoseno,"Reza Rahadian, Bunga Citra Lestari, Tara Basro...",Indonesia,05-Jan-19,2016,TV-PG,124,"Dramas, International Movies, Sports Movies"
3,1000,Movie,Blue Mountain State: The Rise of Thadland,Lev L. Spiro,"Alan Ritchson, Darin Brooks, James Cade, Rob R...",United States,01-Mar-16,2016,R,90,Comedies
4,1001,TV Show,Blue Planet II,Unknown,David Attenborough,United Kingdom,03-Dec-18,2017,TV-G,1,"British TV Shows, Docuseries, Science & Nature TV"


<h1>Generating Synthetic Dataset with customer_id column</h1> Duplicated 10,000 Rows randomly and added customer_id column randomly from range 1 -50

In [13]:
# Create a list of indices to duplicate
indices_to_duplicate = np.random.randint(0, 7787, 2000)

# Duplicate the rows
df_duplicated = df.iloc[indices_to_duplicate]

# Concatenate the original DataFrame and the duplicated rows
df_expanded = pd.concat([df, df_duplicated])
df_expanded.shape

(9787, 11)

In [14]:
# Generate random customer IDs between 1 and 50
customer_ids = np.random.randint(1, 51, size=len(df_expanded))

# Add the 'customer_id' column to the DataFrame
df_expanded['customer_id'] = customer_ids
df_expanded = df_expanded.sort_values(by=['customer_id','show_id'])
df_expanded.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genres,customer_id
2778,35,Movie,#Roxy,Michael Kennedy,"Jake Short, Sarah Fisher, Booboo Stewart, Dann...",Canada,10-Apr-19,2018,TV-14,105,"Comedies, Romantic Movies",1
5778,62,TV Show,12 Years Promise,Unknown,"So-yeon Lee, Namkoong Min, Tae-im Lee, So-hui ...",South Korea,22-May-17,2014,TV-14,1,"International TV Shows, Korean TV Shows, Roman...",1
6555,69,Movie,14 Blades,Daniel Lee,"Donnie Yen, Zhao Wei, Wu Chun, Law Kar-Ying, K...","Hong Kong, China, Singapore",03-Apr-19,2010,R,113,"Action & Adventure, International Movies",1
1545,239,Movie,A Thin Line Between Love & Hate,Martin Lawrence,"Martin Lawrence, Lynn Whitfield, Regina King, ...",United States,01-Dec-20,1996,R,108,"Comedies, Romantic Movies",1
1545,239,Movie,A Thin Line Between Love & Hate,Martin Lawrence,"Martin Lawrence, Lynn Whitfield, Regina King, ...",United States,01-Dec-20,1996,R,108,"Comedies, Romantic Movies",1


<h1>Creating User_Show_Matrix</h1> Rows indicate the customer_id and columns indicate each show_id and the value 1 at particular show_id indicates that particular customer has watched that show and 0 indicates that particular customer hasn't watched that show

In [15]:
customer_show = df_expanded.groupby(['customer_id','show_id'])['show_id'].count().reset_index(name ='Count')
customer_show

,customer_id,show_id,Count
0,1,35,1
1,1,62,1
2,1,69,1
3,1,239,2
4,1,301,1
...,...,...,...
9740,50,7551,1
9741,50,7623,1
9742,50,7625,1
9743,50,7697,1


In [16]:
my_show = customer_show.pivot_table(index='customer_id', columns='show_id', values='Count', aggfunc='sum').fillna(0)
# making a function which returns 0 or 1
# 0 means item was not in that transaction, 1 means item present in that transaction

def encode(x):
    if x<=0:
        return 0
    if x>=1:
        return 1

# applying the function to the dataset
my_show_sets = my_show.map(encode)
my_show_sets.head()

show_id,1,2,3,4,5,6,7,8,9,10,...,7778,7779,7780,7781,7782,7783,7784,7785,7786,7787
customer_id,,,,,,,,,,,,,,,,,,,,,
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<h1>Identifying Frequently Watched Show Combinations</h1>  The apriori algorithm is used to discover these frequent itemsets, and the results are sorted by support to highlight the most commonly watched combinations.

In [17]:
frequent_itemsets = apriori(my_show_sets, min_support=0.03, use_colnames=True)  # Adjust min_support as needed
frequent_itemsets.sort_values(by='support',ascending=False)

C:\Users\keert\AppData\Local\Programs\Python\Python311\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


,support,itemsets
88,0.10,(367)
516,0.10,(2281)
1207,0.08,(5616)
1074,0.08,(4953)
91,0.08,(385)
...,...,...
5508,0.04,"(7072, 4408, 3544, 6006, 7383, 2712)"
5509,0.04,"(7072, 4408, 2897, 6006, 7383, 3544)"
5510,0.04,"(7393, 7458, 3177, 6924, 5430, 3930)"
5511,0.04,"(5702, 6346, 7723, 5935, 399, 1209, 4091)"


In [18]:
df_expanded['show_id'].value_counts().sort_values(ascending=False)

show_id
2281    5
367     5
699     4
5616    4
2410    4
       ..
4691    1
4721    1
4727    1
4841    1
1679    1
Name: count, Length: 7787, dtype: int64

<h1>Discovering Strong Associations Between Shows</h1>  Generate association rules between frequently watched shows

In [19]:
rules = association_rules(frequent_itemsets, metric='lift', min_threshold=1)
rules.sort_values('confidence', ascending = False, inplace = True)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
24208,(2897),"(7072, 3544, 2712, 6006, 7383, 4408)",0.04,0.04,0.04,1.0,25.0,0.0384,inf,1.0
0,(650),(4),0.04,0.04,0.04,1.0,25.0,0.0384,inf,1.0
1,(4),(650),0.04,0.04,0.04,1.0,25.0,0.0384,inf,1.0
2,(1995),(13),0.04,0.04,0.04,1.0,25.0,0.0384,inf,1.0
3,(13),(1995),0.04,0.04,0.04,1.0,25.0,0.0384,inf,1.0


<h1>Content Based Recommendation</h1> Extracts recommended shows from the rules and returns the top N

In [20]:
def recommend_shows(show_id, rules, user_item_matrix, top_n=5):
  """
  Recommends shows based on a given show ID.

  Args:
      show_id (int): The ID of the show to use for recommendations.
      rules (pandas.DataFrame): The association rules generated from Apriori.
      user_item_matrix (pandas.DataFrame): The user-item matrix.
      top_n (int, optional): The number of recommendations to return. Defaults to 5.

  Returns:
      list: A list of recommended show IDs.
  """
  itemsets = rules[rules['antecedents'] == frozenset([show_id])]
  #print(itemsets)
  recommendations = []
  for _, row in itemsets.iterrows():
      recommendations.extend(list(row['consequents']))
  #print(recommendations)
  recommendations = list(set(recommendations) - set([show_id]))[:top_n]
  return recommendations

<h1>Top 5 Content Recommendations</h1>

In [21]:
content_recommendation=recommend_shows(4607, rules, my_show_sets, top_n=10)
print("Top 5 Recommendation Show based on your watch")
content_recommendation

Top 5 Recommendation Show based on your watch


[]

In [22]:
# Assuming you have a DataFrame `df` with 'show_id' and other relevant columns

# ... (rest of your code)

# Create a new DataFrame with the recommended show IDs
recommended_df = pd.DataFrame({'show_id': content_recommendation})

# You can now print or use this DataFrame as needed
print(recommended_df)
# Merge the recommended DataFrame with the original DataFrame
merged_df = pd.merge(recommended_df, df_expanded, on='show_id')

merged_df

Empty DataFrame
Columns: [show_id]
Index: []


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genres,customer_id


In [23]:
df_expanded[df_expanded['show_id']==4607]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genres,customer_id
4009,4607,Movie,Olmo & the Seagull,"Petra Costa, Lea Glob","Olivia Corsini, Sergio Nicolai, Arman Saribeky...","Denmark, Brazil, France, Portugal, Sweden",01-Sep-19,2014,TV-MA,82,"Dramas, Independent Movies, International Movies",6


<h1>Collaborative Recommendations</h1>  computes the pairwise cosine similarity between users 

In [24]:
# Calculate cosine similarity
user_similarity = cosine_similarity(my_show_sets)
print(user_similarity.shape)
user_similarity

(50, 50)


array([[1.        , 0.01064794, 0.01444714, ..., 0.00484889, 0.01020408,
        0.00498879],
       [0.01064794, 1.        , 0.02010076, ..., 0.01011961, 0.00532397,
        0.00520579],
       [0.01444714, 0.02010076, 1.        , ..., 0.00915353, 0.00963143,
        0.00470882],
       ...,
       [0.00484889, 0.01011961, 0.00915353, ..., 1.        , 0.00969777,
        0.        ],
       [0.01020408, 0.00532397, 0.00963143, ..., 0.00969777, 1.        ,
        0.        ],
       [0.00498879, 0.00520579, 0.00470882, ..., 0.        , 0.        ,
        1.        ]])

<p><b>Identifies top similar users and recommends shows they've watched, excluding those already seen by the target user</b></p>

In [25]:
def recommend_similar_user_shows(user_id, user_item_matrix, user_similarity, top_n=10):
    # Get the similarity scores of the target user with all other users
    similar_users = user_similarity[user_id]

    # Sort the users by similarity score in descending order
    similar_users = similar_users.argsort()[::-1]
    print( similar_users)
    # Get the top N similar users
    top_users = similar_users[1:top_n+1]

    # Get the shows watched by the top N similar users
    recommended_shows = []
    for user_id in top_users:
        recommended_shows.extend(user_item_matrix.columns[user_item_matrix.loc[user_id] == 1])

    # Remove shows already watched by the target user
    recommended_shows = list(set(recommended_shows) - set(user_item_matrix.columns[user_item_matrix.loc[user_id] == 1]))

    return recommended_shows[:top_n]

<h1>Top 5 Collabarative Recommendations</h1>

In [26]:
collabarative_recommendation=recommend_similar_user_shows(1,my_show_sets,user_similarity)
print("Top 5 Recommendation Show based on Similar User")
collabarative_recommendation

[ 1  6 21 45  2 22 12 18  9 41 37 30 29 17 39 13 11 15  0 35 32 25 23 10
  8 16 47 42 27 46 20  3  7 34 28 38 48 49 33 31 43 40 36 44 26 24 19 14
  4  5]
Top 5 Recommendation Show based on Similar User


[4096, 1, 3, 5, 4105, 12, 15, 17, 4114, 19]

In [27]:
# Assuming you have a DataFrame `df` with 'show_id' and other relevant columns

# ... (rest of your code)

# Create a new DataFrame with the recommended show IDs
recommended_df = pd.DataFrame({'show_id': collabarative_recommendation})

# You can now print or use this DataFrame as needed
print(recommended_df)
# Merge the recommended DataFrame with the original DataFrame
merged_df = pd.merge(recommended_df, df_expanded, on='show_id')

merged_df

   show_id
0     4096
1        1
2        3
3        5
4     4105
5       12
6       15
7       17
8     4114
9       19


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genres,customer_id
0,4096,Movie,Mike Epps: Only One Mike,Kevin Bray,Mike Epps,United States,25-Jun-19,2019,TV-MA,63,Stand-Up Comedy,9
1,4096,Movie,Mike Epps: Only One Mike,Kevin Bray,Mike Epps,United States,25-Jun-19,2019,TV-MA,63,Stand-Up Comedy,29
2,1,TV Show,3%,Unknown,"João Miguel, Bianca Comparato, Michel Gomes, R...",Brazil,14-Aug-20,2020,TV-MA,4,"International TV Shows, TV Dramas, TV Sci-Fi &...",18
3,3,Movie,23:59,Gilbert Chan,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Singapore,20-Dec-18,2011,R,78,"Horror Movies, International Movies",2
4,5,Movie,21,Robert Luketic,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",United States,01-Jan-20,2008,PG-13,123,Dramas,41
5,4105,TV Show,Million Pound Menu,Unknown,Unknown,United Kingdom,27-Aug-19,2019,TV-MA,2,"British TV Shows, International TV Shows, Real...",7
6,4105,TV Show,Million Pound Menu,Unknown,Unknown,United Kingdom,27-Aug-19,2019,TV-MA,2,"British TV Shows, International TV Shows, Real...",21
7,12,TV Show,1983,Unknown,"Robert Więckiewicz, Maciej Musiał, Michalina O...","Poland, United States",30-Nov-18,2018,TV-MA,1,"Crime TV Shows, International TV Shows, TV Dramas",18
8,15,Movie,3022,John Suits,"Omar Epps, Kate Walsh, Miranda Cosgrove, Angus...",United States,19-Mar-20,2019,R,91,"Independent Movies, Sci-Fi & Fantasy, Thrillers",6
9,15,Movie,3022,John Suits,"Omar Epps, Kate Walsh, Miranda Cosgrove, Angus...",United States,19-Mar-20,2019,R,91,"Independent Movies, Sci-Fi & Fantasy, Thrillers",39


In [39]:
df_expanded[df_expanded['customer_id']==1]

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,genres,customer_id
2778,35,Movie,#Roxy,Michael Kennedy,"Jake Short, Sarah Fisher, Booboo Stewart, Dann...",Canada,10-Apr-19,2018,TV-14,105,"Comedies, Romantic Movies",1
5778,62,TV Show,12 Years Promise,Unknown,"So-yeon Lee, Namkoong Min, Tae-im Lee, So-hui ...",South Korea,22-May-17,2014,TV-14,1,"International TV Shows, Korean TV Shows, Roman...",1
6555,69,Movie,14 Blades,Daniel Lee,"Donnie Yen, Zhao Wei, Wu Chun, Law Kar-Ying, K...","Hong Kong, China, Singapore",03-Apr-19,2010,R,113,"Action & Adventure, International Movies",1
1545,239,Movie,A Thin Line Between Love & Hate,Martin Lawrence,"Martin Lawrence, Lynn Whitfield, Regina King, ...",United States,01-Dec-20,1996,R,108,"Comedies, Romantic Movies",1
1545,239,Movie,A Thin Line Between Love & Hate,Martin Lawrence,"Martin Lawrence, Lynn Whitfield, Regina King, ...",United States,01-Dec-20,1996,R,108,"Comedies, Romantic Movies",1
...,...,...,...,...,...,...,...,...,...,...,...,...
7009,7307,TV Show,Ultramarine Magmell,Unknown,"Kengo Kawanishi, M・A・O, Toshiyuki Morikawa, Hi...",Japan,10-Oct-19,2019,TV-14,1,"Anime Series, International TV Shows",1
7162,7445,Movie,Voulez-vous rire avec moi ce soir?,Yacine Belhousse,"Gad Elmaleh, Eddie Izzard, Bill Burr, Todd Bar...",France,01-Mar-20,2018,TV-MA,93,"Documentaries, International Movies",1
7205,7484,TV Show,Warrior,Unknown,"Dar Salim, Danica Curcic, Lars Ranthe, Marco I...",Denmark,13-Nov-18,2018,TV-MA,1,"Crime TV Shows, International TV Shows, TV Dramas",1
7260,7533,Movie,Well Done Abba,Shyam Benegal,"Boman Irani, Minissha Lamba, Sammir Dattani, I...",India,19-Nov-20,2009,TV-14,135,"Comedies, International Movies",1


<h1>Hybrid Model based on weightage, both collabarative and content based Recommendation</h1>

In [29]:
print("Total Recommendations based on both Collaborative and Content based Recommendations")
content_wt=6
print(content_recommendation)
print(collabarative_recommendation)
wt_content_recommendation=content_recommendation[0:content_wt]
wt_content_recommendation.extend(collabarative_recommendation[0:(10-content_wt)])
print(wt_content_recommendation)

Total Recommendations based on both Collaborative and Content based Recommendations
[]
[4096, 1, 3, 5, 4105, 12, 15, 17, 4114, 19]
[4096, 1, 3, 5]


In [43]:
hybrid_recommended_df = pd.DataFrame({'show_id': collabarative_recommendation})

# You can now print or use this DataFrame as needed
#print(hybrid_recommended_df)
# Merge the recommended DataFrame with the original DataFrame
Recommendation_df = pd.merge(hybrid_recommended_df, df_expanded, on='show_id')
Recommendation_df.drop('customer_id', axis=1,inplace=True)
Recommendation_df=Recommendation_df.drop_duplicates()
Recommendation_df[['title', 'cast','director','genres']]

,title,cast,director,genres
0,Mike Epps: Only One Mike,Mike Epps,Kevin Bray,Stand-Up Comedy
2,3%,"João Miguel, Bianca Comparato, Michel Gomes, R...",Unknown,"International TV Shows, TV Dramas, TV Sci-Fi &..."
3,23:59,"Tedd Chan, Stella Chung, Henley Hii, Lawrence ...",Gilbert Chan,"Horror Movies, International Movies"
4,21,"Jim Sturgess, Kevin Spacey, Kate Bosworth, Aar...",Robert Luketic,Dramas
5,Million Pound Menu,Unknown,Unknown,"British TV Shows, International TV Shows, Real..."
7,1983,"Robert Więckiewicz, Maciej Musiał, Michalina O...",Unknown,"Crime TV Shows, International TV Shows, TV Dramas"
8,3022,"Omar Epps, Kate Walsh, Miranda Cosgrove, Angus...",John Suits,"Independent Movies, Sci-Fi & Fantasy, Thrillers"
10,09-Feb,"Shahd El Yaseen, Shaila Sabt, Hala, Hanadi Al-...",Unknown,"International TV Shows, TV Dramas"
11,Mini Wolf,Unknown,Unknown,Kids' TV
12,15-Aug,"Rahul Pethe, Mrunmayee Deshpande, Adinath Koth...",Swapnaneel Jayakar,"Comedies, Dramas, Independent Movies"
